In [31]:
import fastf1
round_num = 1
season = 2022
segment = 1
session = fastf1.get_session(year=season, gp=round_num, identifier=segment)

#TODO
#Event details (weekend, location, schedules)
#Driver details (name, team, contract, number, born_on)
#Constructor details (name, titles_won, years_since_last_title, years_since_last_win, engine_maker?)
#Circuit details (name, location, length, turns, lap_record)
#Segment details (segment, start_time, end_time, duration, weather, track_temp, air_temp, humidity, wind_speed, wind_direction)
#Telemetry

In [35]:
fastf1.get_testing_event(year=season, test_number=1)

RoundNumber                                                0
Country                                                Spain
Location                                               Spain
OfficialEventName    FORMULA 1 PRE-SEASON TRACK SESSION 2022
EventDate                                2022-02-25 00:00:00
EventName                           Pre-Season Track Session
EventFormat                                          testing
Session1                                          Practice 1
Session1Date                       2022-02-23 09:00:00+01:00
Session1DateUtc                          2022-02-23 08:00:00
Session2                                          Practice 2
Session2Date                       2022-02-24 09:00:00+01:00
Session2DateUtc                          2022-02-24 08:00:00
Session3                                          Practice 3
Session3Date                       2022-02-25 09:00:00+01:00
Session3DateUtc                          2022-02-25 08:00:00
Session4                

In [59]:
#EventGroups
df = pl.from_pandas(fastf1.get_event_schedule(year=season))
df.select(
    pl.concat_str(['RoundNumber', pl.lit('_'), pl.col('EventDate').cast(pl.Date)]).alias('id'),
    pl.col('EventFormat').str.to_lowercase().alias('event_group_type'),
    pl.col('EventDate').dt.year().alias('season'),
    pl.col('Location').str.to_lowercase().alias('circuit_id'),
).head(5)

id,event_group_type,season,circuit_id
str,str,i32,str
"""0_2022-02-25""","""testing""",2022,"""spain"""
"""0_2022-03-12""","""testing""",2022,"""bahrain"""
"""1_2022-03-20""","""conventional""",2022,"""sakhir"""
"""2_2022-03-27""","""conventional""",2022,"""jeddah"""
"""3_2022-04-10""","""conventional""",2022,"""melbourne"""


In [85]:
df.with_columns(conc_sessions=len(pl.concat_str([
        pl.col("Session1").str.slice(-1),
        pl.col("Session2").str.slice(-1)
    ]))).head(5)

TypeError: object of type 'Expr' has no len()

In [72]:
session_columns = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
df.with_columns(
    pl.col("Session1").replace("None", None).is_not_null().alias("Session1"),
).select(session_columns).head(5)

Session1,Session2,Session3,Session4,Session5
bool,str,str,str,str
true,"""Practice 2""","""Practice 3""","""None""","""None"""
true,"""Practice 2""","""Practice 3""","""None""","""None"""
true,"""Practice 2""","""Practice 3""","""Qualifying""","""Race"""
true,"""Practice 2""","""Practice 3""","""Qualifying""","""Race"""
true,"""Practice 2""","""Practice 3""","""Qualifying""","""Race"""


In [64]:
mapper = {"None": None}
pl.from_pandas(fastf1.get_event_schedule(year=season)).map_dict(mapper)

AttributeError: 'DataFrame' object has no attribute 'map_dict'

In [32]:
fastf1.get_event(year=season, gp=round_num)

RoundNumber                                                   1
Country                                                 Bahrain
Location                                                 Sakhir
OfficialEventName    FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022
EventDate                                   2022-03-20 00:00:00
EventName                                    Bahrain Grand Prix
EventFormat                                        conventional
Session1                                             Practice 1
Session1Date                          2022-03-18 15:00:00+03:00
Session1DateUtc                             2022-03-18 12:00:00
Session2                                             Practice 2
Session2Date                          2022-03-18 18:00:00+03:00
Session2DateUtc                             2022-03-18 15:00:00
Session3                                             Practice 3
Session3Date                          2022-03-19 15:00:00+03:00
Session3DateUtc                         

In [30]:
session.event

RoundNumber                                                   1
Country                                                 Bahrain
Location                                                 Sakhir
OfficialEventName    FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022
EventDate                                   2022-03-20 00:00:00
EventName                                    Bahrain Grand Prix
EventFormat                                        conventional
Session1                                             Practice 1
Session1Date                          2022-03-18 15:00:00+03:00
Session1DateUtc                             2022-03-18 12:00:00
Session2                                             Practice 2
Session2Date                          2022-03-18 18:00:00+03:00
Session2DateUtc                             2022-03-18 15:00:00
Session3                                             Practice 3
Session3Date                          2022-03-19 15:00:00+03:00
Session3DateUtc                         

In [18]:
import polars as pl
mydict = {
    'name': session.name
}
pl.DataFrame(mydict)

name
str
"""Practice 1"""
